<center> 
<h3><b> Adnan Rashid </b><\h3>
<br><br>
    <nav>
    | <a href="00-Contents-Setup.ipynb"> Home Page </a> | 
    <a href="http://bit.ly/cka_notes_original"> Kubernetes Notes </a> |
    <a href="https://adnan.study"> Website </a> |
    <a href="https://www.instagram.com/adnans_techie_studies/"> Instagram </a> |
    <a href="https://www.linkedin.com/in/adnanrashid1/"> LinkedIn </a> |
    </nav>
</center>

<h1 style="background:black"><code style="background:black;color:white"><center>Deploying Jenkins</center></code></h1>

<div class="alert alert-block alert-warning">
<font style="color:black">
    <center>In this section, we will deploy Jenkins using the <b> helm </b> package manager </center>      
</font>
</div>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/31.jpg" width="50%" height="50%" border=5/></center>

## Pre-requisite

* Check your variables are loaded

In [1]:
%store

Stored variables and their in-db values:
ACCOUNT_ID                     -> 133150622354
AWS_DEFAULT_REGION             -> 'eu-west-1'
AWS_PROFILE                    -> 'gr-sandbox-admin'
MASTER_ARN                     -> 'arn:aws:kms:eu-west-1:133150622354:key/a4094ca3-d
ROLE_NAME                      -> 'eksctl-eksworkshop-eksctl-nodegro-NodeInstanceRol
STACK_NAME                     -> 'eksctl-eksworkshop-eksctl-nodegroup-nodegroup'


* Load in variable store

In [ ]:
%store -r 

* Load variables into notebook

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env AWS_DEFAULT_REGION=$AWS_DEFAULT_REGION
%env ACCOUNT_ID=$ACCOUNT_ID

* Check identity 

In [ ]:
!aws sts get-caller-identity

<br>

---
---

<br>

With our Storage Class configured we then need to crate our <b> jenkins </b> setup. To do this we'll just use the <b> helm </b> cli with a few flags. 

In a production system you should be using the values.yaml file so that you can manage the drift as you need to update releases. 

<br>

* Install Jenkins Helm Chart

In [ ]:
!helm install cicd stable/jenkins --set rbac.create=true,master.servicePort=80,master.serviceType=LoadBalancer

<br>
The output of this command will give ou some additional infromation such as the <b> admin </b> password and the way to get the host name of the ELB that was provisioned. 

* Check the pods, this can take around 5 minutes or you should see it go from init, pending to running state

In [ ]:
!kubectl get pods

* Once it is in running state we can get the load balancer address

In [ ]:
%%bash 
export SERVICE_IP=$(kubectl get svc --namespace default cicd-jenkins --template "{{ range (index .status.loadBalancer.ingress 0) }}{{ . }}{{ end }}")
echo http://$SERVICE_IP/login

* Here we can login with the username 'admin' and the password running the below

In [ ]:
!printf $(kubectl get secret --namespace default cicd-jenkins -o jsonpath="{.data.jenkins-admin-password}" | base64 --decode);echo

## <br><br> Clean Up 

In [ ]:
!helm uninstall cicd